In [1]:
import warnings
import numpy as np

from keras.models import Model
from keras import layers
from keras.layers import Activation, Dropout, Input, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Dense
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K

from keras.utils.data_utils import get_file
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.engine.topology import get_source_inputs

E:\AISTUDYING\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def Conv_bn(inputs, 
            filters, 
            kernel = (3,3), 
            strides = (1,1), 
            padding = 'same', name = None):
    if name is not None:
        bn_name = name +'_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    x = Conv2D(filters, kernel, strides = strides, padding = padding,
               use_bias =False,
              name = conv_name)(inputs)
    x = BatchNormalization(scale = False, name = bn_name)(x)
    x = Activation('relu', name = name)(x)
    
    return x
    
def InceptionV3 (input_shape = [299,299,3], num_class = 1000):
    
    img_input = Input(shape = input_shape)
    
    x = Conv_bn(img_input,32, kernel = (3,3), strides = (2,2), padding = 'valid')
    print(x)
    x = Conv_bn(x,32, kernel = (3,3), padding = 'valid')
    print(x)
    x = Conv_bn(x,64, kernel = (3,3))
    print(x)
    x =MaxPooling2D((3,3), strides = (2,2))(x)
    print(x)
    
    x = Conv_bn(x,80, kernel = (1,1),  padding = 'valid')
    print(x)
    x = Conv_bn(x,192, kernel = (3,3), padding = 'valid')
    print(x)
    x = MaxPooling2D((3,3), strides = (2,2))(x)
    print(x)
    
    '''
    block1 
    1X1Conv
    1X1Conv - 3X3Conv
    1X1Conv - 3X3Conv - 3X3Conv
    AvgPooling-1x1Conv
    '''
    #block1  part1
    #输入35X35X192, 输出35X35X288  # 64+64+96+32 = 256  nhwc-0123
    branch1x1 = Conv_bn(x,64, kernel = (1,1))
    
    branch5x5 = Conv_bn(x,48, kernel = (1,1))
    branch5x5 = Conv_bn(branch5x5,64, kernel = (5,5) )
    
    branch3x3db1 = Conv_bn(x,64, kernel = (1,1) )
    branch3x3db1 = Conv_bn(branch3x3db1,96, kernel = (3,3))
    branch3x3db1 = Conv_bn(branch3x3db1,96, kernel = (3,3))
    
    branch_pool = AveragePooling2D((3,3), strides = (1,1), padding = 'same')(x)
    branch_pool = Conv_bn(branch_pool,32, kernel = (1,1))
    
    x = layers.concatenate([branch1x1, branch5x5, branch3x3db1, branch_pool], axis = 3,
                          name = 'mixedd0')
    #block1  part2
    #输入35X35X256, 输出35X35X288
    branch1x1 = Conv_bn(x,64, kernel = (1,1) )
    
    branch5x5 = Conv_bn(x,48, kernel = (1,1) )
    branch5x5 = Conv_bn(branch5x5,64, kernel = (5,5))
    
    branch3x3db1 = Conv_bn(x,64, kernel = (1,1) )
    branch3x3db1 = Conv_bn(branch3x3db1,96, kernel = (3,3) )
    branch3x3db1 = Conv_bn(branch3x3db1,96, kernel = (3,3) )
    

    branch_pool = AveragePooling2D((3,3), strides = (1,1), padding = 'same')(x)
    branch_pool = Conv_bn(branch_pool,64, kernel = (1,1))
    
    x = layers.concatenate([branch1x1, branch5x5, branch3x3db1, branch_pool], axis = 3,
                          name = 'mixedd1')
    print(x)
    #block1  part3
    #输入35X35X288, 输出35X35X288
    branch1x1 = Conv_bn(x,64, kernel = (1,1))
    
    branch5x5 = Conv_bn(x,48, kernel = (1,1) )
    branch5x5 = Conv_bn(branch5x5,64, kernel = (5,5) )
    
    branch3x3db1 = Conv_bn(x,64, kernel = (1,1))
    branch3x3db1 = Conv_bn(branch3x3db1,96, kernel = (3,3) )
    branch3x3db1 = Conv_bn(branch3x3db1,96, kernel = (3,3) )
    
    branch_pool = AveragePooling2D((3,3), strides = (1,1), padding = 'same')(x)
    branch_pool = Conv_bn(branch_pool,64, kernel = (1,1))
    
    x = layers.concatenate([branch1x1, branch5x5, branch3x3db1, branch_pool], axis = 3,
                          name = 'mixedd2')
    print(x)    
    #block2  part1
    #输入35X35X288, 输出17X17X768  
    branch3x3 = Conv_bn(x,384, kernel = (3,3), strides = (2,2), padding = 'valid')
    
    branch5x5 = Conv_bn(x,64, kernel = (1,1))
    branch5x5 = Conv_bn(branch5x5, 96, kernel = (3,3))
    branch5x5 = Conv_bn(branch5x5, 96, kernel = (3,3), strides = (2,2), padding = 'valid')
    
    branch_pool = MaxPooling2D((3,3), strides = (2,2))(x)
    
    x = layers.concatenate([branch3x3, branch5x5, branch_pool], axis = 3,
                          name = 'mixedd3')
    print(x)
    #block2  part2
    #输入17X17X768 , 输出17X17X768 
    branch1x1 = Conv_bn(x,192, kernel = (1,1))
    
    branch7x7 = Conv_bn(x,128, kernel = (1,1))
    branch7x7 = Conv_bn(branch7x7, 128, kernel = (1,7))
    branch7x7 = Conv_bn(branch7x7, 192, kernel = (7,1))
    
    branch13x13 = Conv_bn(x,128, kernel = (1,1))
    branch13x13 = Conv_bn(branch13x13, 128, kernel = (7,1))
    branch13x13 = Conv_bn(branch13x13, 128, kernel = (1,7))
    branch13x13 = Conv_bn(branch13x13, 128, kernel = (7,1))
    branch13x13 = Conv_bn(branch13x13, 192, kernel = (1,7))
    
    branch_pool = AveragePooling2D((3,3),strides = (1,1), padding = 'same')(x)
    branch_pool = Conv_bn(branch_pool,192, kernel = (1,1))
    
    x = layers.concatenate([branch1x1, branch7x7,branch13x13, branch_pool], axis = 3,
                          name = 'mixedd4') 
    print(x)
    #block2  part3/4
    #输入17X17X768 , 输出17X17X768     
    for i in range(2):
        branch1x1 = Conv_bn(x,192, kernel = (1,1))
    
        branch7x7 = Conv_bn(x,160, kernel = (1,1))
        branch7x7 = Conv_bn(branch7x7, 160, kernel = (1,7))
        branch7x7 = Conv_bn(branch7x7, 192, kernel = (7,1))
    
        branch13x13 = Conv_bn(x,160, kernel = (1,1))
        branch13x13 = Conv_bn(branch13x13, 160, kernel = (7,1))
        branch13x13 = Conv_bn(branch13x13, 160, kernel = (1,7))
        branch13x13 = Conv_bn(branch13x13, 160, kernel = (7,1))
        branch13x13 = Conv_bn(branch13x13, 192, kernel = (1,7))
    
        branch_pool = AveragePooling2D((3,3),strides = (1,1), padding = 'same')(x)
        branch_pool = Conv_bn(branch_pool,192, kernel = (1,1))
    
        x = layers.concatenate([branch1x1, branch7x7,branch13x13, branch_pool], axis = 3,
                              name = 'mixed%d' % (i+5) )
        
    print(x)
    #block2  part3/4
    #输入17X17X768 , 输出17X17X768   
    branch1x1 = Conv_bn(x,192, kernel = (1,1))
    
    branch7x7 = Conv_bn(x,192, kernel = (1,1))
    branch7x7 = Conv_bn(branch7x7, 192, kernel = (1,7))
    branch7x7 = Conv_bn(branch7x7, 192, kernel = (7,1))
    
    branch13x13 = Conv_bn(x,192, kernel = (1,1))
    branch13x13 = Conv_bn(branch13x13, 192, kernel = (7,1))
    branch13x13 = Conv_bn(branch13x13, 192, kernel = (1,7))
    branch13x13 = Conv_bn(branch13x13, 192, kernel = (7,1))
    branch13x13 = Conv_bn(branch13x13, 192, kernel = (1,7))
    
    branch_pool = AveragePooling2D((3,3),strides = (1,1), padding = 'same')(x)
    branch_pool = Conv_bn(branch_pool,192, kernel = (1,1))
    
    x = layers.concatenate([branch1x1, branch7x7,branch13x13, branch_pool], axis = 3,
                          name = 'mixedd7')  
    print(x)    
    #block3  part1
    #输入17X17X768 , 输出8X8X1280    
    branch3x3 = Conv_bn(x,192, kernel = (1,1))    
    branch3x3 = Conv_bn(branch3x3,320, kernel = (3,3), strides = (2,2), padding = 'valid')
    
    branch7x7 = Conv_bn(x,192, kernel = (1,1))    
    branch7x7 = Conv_bn(branch7x7,192, kernel = (1,7))    
    branch7x7 = Conv_bn(branch7x7,192, kernel = (7,1))  
    branch7x7 = Conv_bn(branch7x7,192, kernel = (3,3), strides = (2,2), padding = 'valid')  
    
    branch_pool = MaxPooling2D((3,3), strides = (2,2))(x)    
    
    x = layers.concatenate([branch3x3, branch7x7,branch_pool], axis = 3, name = 'mixedd8')      
    print(x)    
    #block3  part2
    #输出8X8X1280   , 输出8X8X2048
    for i in range(2):
        
        branch1x1 = Conv_bn(x,320, kernel = (1,1))
    
        branch3x3a = Conv_bn(x,384, kernel = (1,1))
        branch3x3b = Conv_bn(branch3x3a,384, kernel = (1,3))
        branch3x3c = Conv_bn(branch3x3a,384, kernel = (3,1))

        branch3x3 = layers.concatenate([branch3x3b,branch3x3c], axis = 3, name='mixed9_' + str(i)) 
    
    
        branch7x7 = Conv_bn(x,448, kernel = (1,1))
        branch7x7 = Conv_bn(branch7x7,384, kernel = (3,3))
        branch7x7a = Conv_bn(branch7x7, 384, kernel = (1,3))
        branch7x7b = Conv_bn(branch7x7, 384, kernel = (3,1))
        
        branch7x7 = layers.concatenate([branch7x7a,branch7x7b], axis = 3) 
    
        branch_pool = AveragePooling2D((3,3), strides = (1,1), padding = 'same')(x)
        branch_pool = Conv_bn(branch_pool,192, kernel = (1,1))
    
        x = layers.concatenate([branch1x1, branch3x3,branch7x7, branch_pool], axis = 3,
                              name =  'mixed%d' % (i+9))     
        print(x)      
    #平均池化层之后全连接层
    x = GlobalAveragePooling2D(name = 'avgpool')(x)
    x = Dense(num_class,activation = 'softmax', name = 'prediction' )(x)
    
    inputs = img_input
    model = Model(inputs, x, name = 'Inceptionv3')
    
    return model

In [3]:
def preprocessing_input(x):
    x /= 255.
    x -= 0.5
    x *= 2
    return x

In [4]:
if __name__ == '__main__':
    model = InceptionV3()
    try:
        model.load_weights("inception_v3_weights_tf_dim_ordering_tf_kernels.h5")
        print('成功加载权重')
    except ValueError as e:
        print('加载权重失败')
        model.summary()
        
    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size = (299,299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocessing_input(img)
    
    prediction = model.predict(img)
    print('prediction', np.argmax(prediction))
    print('prediction', decode_predictions(prediction))

Tensor("activation_1/Relu:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("activation_2/Relu:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("activation_3/Relu:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 73, 73, 64), dtype=float32)
Tensor("activation_4/Relu:0", shape=(?, 73, 73, 80), dtype=float32)
Tensor("activation_5/Relu:0", shape=(?, 71, 71, 192), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 35, 35, 192), dtype=float32)
Tensor("mixedd1/concat:0", shape=(?, 35, 35, 288), dtype=float32)
Tensor("mixedd2/concat:0", shape=(?, 35, 35, 288), dtype=float32)
Tensor("mixedd3/concat:0", shape=(?, 17, 17, 768), dtype=float32)
Tensor("mixedd4/concat:0", shape=(?, 17, 17, 768), dtype=float32)
Tensor("mixed6/concat:0", shape=(?, 17, 17, 768), dtype=float32)
Tensor("mixedd7/concat:0", shape=(?, 17, 17, 768), dtype=float32)
Tensor("mixedd8/concat:0", shape=(?, 8, 8, 1280), dtype=float32)
Tensor("mixed9/concat:0", shape=(?, 8, 8, 20